In [ ]:

import numpy as np
import pandas as pd
import warnings
from datetime import datetime
from QuantConnect.Research import QuantBook
from QuantConnect import Resolution, TimeZones, DataNormalizationMode
from statsmodels.tsa.stattools import pacf
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt  # Para graficar la FFT
import quantstats as qs
from itertools import product
# Ignorar advertencias de métricas no definidas
warnings.simplefilter("ignore", category=UserWarning)
# Inicializa QuantBook
qb = QuantBook()
weekdays = range(0, 7)  # Días de la semana (0=Lunes, 6=Domingo)
days = range(1, 32)  # Días del mes (1-31)
all_best_seasons = []
count = 0
for symbol in symbols:
    count += 1
    print(count, symbol)
    asset = qb.AddEquity(symbol, Resolution.DAILY, dataNormalizationMode=DataNormalizationMode.RAW).Symbol
    history = qb.History(asset, start=datetime(1997, 1, 1), end=qb.time, resolution=Resolution.DAILY)
    if isinstance(history.index, pd.MultiIndex):
        history = history.droplevel([0])
    if not history.empty:
        if history.index.min().year > 2007 or history.index.max().year < 2025:
            print(f"Datos insuficientes para {symbol}, descartando...")
            continue
        history = history[['open', 'high', 'low', 'close', 'volume']].copy()
        # Eliminar filas donde el volumen sea 0
        history['returns_pct'] = ((history['close'] - history['open']) / history['open']) * 100
        history['pct_range'] = abs((history['high'] - history['low']) / history['low']) * 100
        history['day'] = history.index.get_level_values('time').day
        history['weekday'] = history.index.get_level_values('time').weekday
        up_trend = (history['close'].shift(1) > history['close'].shift(2))
        up_breakout = (history['close'].shift(1) > history['high'].shift(2))
        down_trend = (history['close'].shift(1) < history['close'].shift(2))
        down_breakout = (history['close'].shift(1) < history['low'].shift(2))
        periods = {"day": days, "weekday": weekdays}
        for period, values in periods.items():
            for value in values:
                filtro = history[history[period] == value].copy()
                if filtro.empty:
                    continue
                pnls = filtro['returns_pct']
                num_trades = len(pnls)
                gains = pnls[pnls > 0].sum()
                losses = abs(pnls[pnls < 0].sum())
                pf_longs = gains / losses if losses != 0 else float('inf')
                sharpe_ratio = qs.stats.sharpe(pnls) if len(pnls) > 1 else float('inf')
                all_best_seasons.append([symbol, period, value, pf_longs, sharpe_ratio, num_trades])
    else:
        print(f"No hay datos disponibles para {symbol}")

best_seasons_df = pd.DataFrame(all_best_seasons, columns=["Symbol", "Period", "Value", "ProfitFactor", "SharpeRatio", "NumTrades"])
best_seasons_df = best_seasons_df.sort_values(by=["ProfitFactor"], ascending=False)




In [ ]:

filtered_seasons = best_seasons_df[
    (best_seasons_df['Period'] == 'weekday') &
    (best_seasons_df["ProfitFactor"] > 1.5) &
    (best_seasons_df["ProfitFactor"] < 5.0) &
    (best_seasons_df["SharpeRatio"] > 1.5) &
    (best_seasons_df["NumTrades"] > 100)
].sort_values(by="ProfitFactor", ascending=False)  # Ordenado por Period y luego por Value

print(filtered_seasons)


In [ ]:

import logging
import matplotlib.font_manager
import pandas as pd
import matplotlib.pyplot as plt
import quantstats as qs

logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

long = True

# Iterar sobre cada combinación filtrada en orden cronológico
for index, row in filtered_seasons.iterrows():
    symbol = row["Symbol"]
    period = row["Period"]
    value = row["Value"]

    asset = qb.AddEquity(symbol, Resolution.DAILY, dataNormalizationMode=DataNormalizationMode.RAW).Symbol
    history = qb.History(asset, start=datetime(1997, 1, 1), end=qb.time, resolution=Resolution.DAILY)
    if isinstance(history.index, pd.MultiIndex):
        history = history.droplevel([0])

    if not history.empty:
        history = history[['open', 'high', 'low', 'close', 'volume']].copy()

        history['returns_pct'] = ((history['close'] - history['open']) / history['open']) * 100
        history['pct_range'] = abs((history['high'] - history['low']) / history['low']) * 100

        if not long:
            history['returns_pct'] *= -1  # Invertir retornos para shorts

        history['month'] = history.index.get_level_values('time').month
        history['day'] = history.index.get_level_values('time').day
        history['weekday'] = history.index.get_level_values('time').weekday
        history['hour'] = history.index.get_level_values('time').hour
        up_trend = (history['close'].shift(1) > history['close'].shift(2))
        up_breakout = (history['close'].shift(1) > history['high'].shift(2))
        down_trend = (history['close'].shift(1) < history['close'].shift(2))
        down_breakout = (history['close'].shift(1) < history['low'].shift(2))

        # Filtrar datos según los valores de period y value de la combinación actual
        filtro = history[history[period] == value].copy()

        # Guardar los retornos filtrados en una serie de pandas
        pnls = pd.Series(filtro['returns_pct'].values, index=filtro.index) / 100
        filtro['equity'] = (1 + filtro['returns_pct'] / 100).cumprod() * 100

        # Generar informe de estrategia
        qs.extend_pandas()
        if pnls.empty:
          print(f"Advertencia: No hay datos suficientes para {symbol} ({period} {value})")
          continue
        pnls.plot_snapshot(title=f'Rendimiento de la Estrategia ({symbol}, {period} {value})', show=True)

        num_trades = len(pnls)
        print("\n" + "="*80)
        print(f"PERFORMANCE REPORT: {symbol} ({period} {value})")
        print("="*80 + "\n")
        print(f"{'Metric':<30}{'Value':>20}")
        print("-"*50)
        print(f"{'Profit Factor':<30}{qs.stats.profit_factor(pnls):>20.2f}")
        print(f"{'Annualized Return (CAGR)':<30}{qs.stats.cagr(pnls):>20.2%}")
        print(f"{'Sharpe Ratio':<30}{qs.stats.sharpe(pnls):>20.2f}")
        print(f"{'Sortino Ratio':<30}{qs.stats.sortino(pnls):>20.2f}")
        print(f"{'Max Drawdown':<30}{qs.stats.max_drawdown(pnls):>20.2%}")
        print(f"{'Value at Risk (VaR 95%)':<30}{qs.stats.value_at_risk(pnls):>20.2%}")
        print(f"{'Conditional VaR (CVaR 95%)':<30}{qs.stats.conditional_value_at_risk(pnls):>20.2%}")
        print(f"{'Recovery Factor':<30}{qs.stats.recovery_factor(pnls):>20.2f}")
        print(f"{'Win Rate':<30}{qs.stats.win_rate(pnls):>20.2%}")
        print(f"{'Skewness':<30}{qs.stats.skew(pnls):>20.2f}")
        print(f"{'Kurtosis':<30}{qs.stats.kurtosis(pnls):>20.2f}")
        print(f"{'Calmar Ratio':<30}{qs.stats.cagr(pnls) / abs(qs.stats.max_drawdown(pnls)):>20.2f}")
        print(f"{'Number of Trades':<30}{num_trades:>20}")
        print("-"*50 + "\n")
        print("="*80 + "\n")


In [ ]:
import logging
import matplotlib.font_manager
import pandas as pd
import matplotlib.pyplot as plt
import quantstats as qs
import numpy as np
from datetime import datetime
from itertools import product

logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

filtered_seasons = best_seasons_df[
    (best_seasons_df['Period'] == 'weekday') &
    (best_seasons_df["ProfitFactor"] > 1.5) &
    (best_seasons_df["SharpeRatio"] > 1.5) &
    (best_seasons_df["SharpeRatio"] < 5.0) &
    (best_seasons_df["NumTrades"] > 100)
].sort_values(by="ProfitFactor", ascending=False)

long = True

tp_values = np.round(np.arange(0.1, 5.1, 0.1), 2)
sl_values = list(np.round(np.arange(0.1, 5.1, 0.1), 2)) + [None]

for index, row in filtered_seasons.iterrows():
    symbol = row["Symbol"]
    period = row["Period"]
    value = row["Value"]

    asset = qb.AddEquity(symbol, Resolution.DAILY, dataNormalizationMode=DataNormalizationMode.RAW).Symbol
    history = qb.History(asset, start=datetime(1997, 1, 1), end=qb.time, resolution=Resolution.DAILY)
    if isinstance(history.index, pd.MultiIndex):
        history = history.droplevel([0])

    if not history.empty:
        history = history[['open', 'high', 'low', 'close', 'volume']].copy()

        history['returns_pct'] = ((history['close'] - history['open']) / history['open']) * 100
        history['pct_range'] = abs((history['high'] - history['low']) / history['low']) * 100
        history['pct_range'] = history['pct_range'].shift(1)

        if not long:
            history['returns_pct'] *= -1

        history['month'] = history.index.get_level_values('time').month
        history['day'] = history.index.get_level_values('time').day
        history['weekday'] = history.index.get_level_values('time').weekday

        filtro = history[history[period] == value].copy()

        results = []

        for tp_pct, sl_pct in product(tp_values, sl_values):
            temp = filtro.copy()

            temp['take_profit_price'] = temp['open'] * (1 + (tp_pct * temp['pct_range'] / 100))
            temp['hit_tp'] = temp['high'] >= temp['take_profit_price']
            temp['returns_pct'] = np.where(temp['hit_tp'], tp_pct * temp['pct_range'], temp['returns_pct'])

            if sl_pct is not None:
                temp['stop_loss_price'] = temp['open'] * (1 - (sl_pct * temp['pct_range'] / 100))
                temp['hit_sl'] = temp['low'] <= temp['stop_loss_price']
                temp['returns_pct'] = np.where(temp['hit_sl'], -sl_pct * temp['pct_range'], temp['returns_pct'])

            returns = temp['returns_pct'] / 100
            equity = (1 + returns).cumprod() * 100

            try:
                sharpe = qs.stats.sharpe(returns)
            except:
                sharpe = np.nan

            results.append({
                'tp_pct': tp_pct,
                'sl_pct': sl_pct,
                'final_equity': equity.iloc[-1] if not equity.empty else 0,
                'sharpe': sharpe,
                'equity_curve': equity,
                'returns_pct': returns
            })

        df_results = pd.DataFrame(results)

        # Mejor resultado con SL basado en Sharpe
        best_with_sl = df_results[df_results['sl_pct'].notnull()].sort_values(by='sharpe', ascending=False).iloc[0]

        # Mejor resultado sin SL basado en Sharpe
        best_without_sl = df_results[df_results['sl_pct'].isnull()].sort_values(by='sharpe', ascending=False).iloc[0]

        print(f"Mejor configuración con SL ({symbol}, {period} {value}):")
        print(f"TP = {best_with_sl['tp_pct']:.2f}, SL = {best_with_sl['sl_pct']:.2f}")
        print(f"Sharpe Ratio: {best_with_sl['sharpe']:.2f}")

        print(f"Mejor configuración sin SL ({symbol}, {period} {value}):")
        print(f"TP = {best_without_sl['tp_pct']:.2f}, SL = Ninguno")
        print(f"Sharpe Ratio: {best_without_sl['sharpe']:.2f}")

        # Mostrar gráfico del mejor caso con SL
        qs.extend_pandas()
        pnl_series = pd.Series(best_with_sl['returns_pct'].values, index=filtro.index[:len(best_with_sl['returns_pct'])])
        pnl_series.plot_snapshot(title=f'Con SL - {symbol} ({period} {value})', show=True)
        num_trades = len(pnl_series)
        print("\n" + "="*80)
        print(f"PERFORMANCE REPORT: {symbol} ({period} {value})")
        print("="*80 + "\n")
        print(f"{'Metric':<30}{'Value':>20}")
        print("-"*50)
        print(f"{'Profit Factor':<30}{qs.stats.profit_factor(pnl_series):>20.2f}")
        print(f"{'Annualized Return (CAGR)':<30}{qs.stats.cagr(pnl_series):>20.2%}")
        print(f"{'Sharpe Ratio':<30}{qs.stats.sharpe(pnl_series):>20.2f}")
        print(f"{'Sortino Ratio':<30}{qs.stats.sortino(pnl_series):>20.2f}")
        print(f"{'Max Drawdown':<30}{qs.stats.max_drawdown(pnl_series):>20.2%}")
        print(f"{'Value at Risk (VaR 95%)':<30}{qs.stats.value_at_risk(pnl_series):>20.2%}")
        print(f"{'Conditional VaR (CVaR 95%)':<30}{qs.stats.conditional_value_at_risk(pnl_series):>20.2%}")
        print(f"{'Recovery Factor':<30}{qs.stats.recovery_factor(pnl_series):>20.2f}")
        print(f"{'Win Rate':<30}{qs.stats.win_rate(pnl_series):>20.2%}")
        print(f"{'Calmar Ratio':<30}{qs.stats.cagr(pnl_series) / abs(qs.stats.max_drawdown(pnl_series)):>20.2f}")
        print(f"{'Number of Trades':<30}{num_trades:>20}")
        print("-"*50 + "\n")
        print("="*80 + "\n")

        # Mostrar gráfico del mejor caso sin SL
        pnl_series_no_sl = pd.Series(best_without_sl['returns_pct'].values, index=filtro.index[:len(best_without_sl['returns_pct'])])
        pnl_series_no_sl.plot_snapshot(title=f'Sin SL - {symbol} ({period} {value})', show=True)
        num_trades = len(pnl_series_no_sl)
        print("\n" + "="*80)
        print(f"PERFORMANCE REPORT: {symbol} ({period} {value})")
        print("="*80 + "\n")
        print(f"{'Metric':<30}{'Value':>20}")
        print("-"*50)
        print(f"{'Profit Factor':<30}{qs.stats.profit_factor(pnl_series_no_sl):>20.2f}")
        print(f"{'Annualized Return (CAGR)':<30}{qs.stats.cagr(pnl_series_no_sl):>20.2%}")
        print(f"{'Sharpe Ratio':<30}{qs.stats.sharpe(pnl_series_no_sl):>20.2f}")
        print(f"{'Sortino Ratio':<30}{qs.stats.sortino(pnl_series_no_sl):>20.2f}")
        print(f"{'Max Drawdown':<30}{qs.stats.max_drawdown(pnl_series_no_sl):>20.2%}")
        print(f"{'Value at Risk (VaR 95%)':<30}{qs.stats.value_at_risk(pnl_series_no_sl):>20.2%}")
        print(f"{'Conditional VaR (CVaR 95%)':<30}{qs.stats.conditional_value_at_risk(pnl_series_no_sl):>20.2%}")
        print(f"{'Recovery Factor':<30}{qs.stats.recovery_factor(pnl_series_no_sl):>20.2f}")
        print(f"{'Win Rate':<30}{qs.stats.win_rate(pnl_series_no_sl):>20.2%}")
        print(f"{'Calmar Ratio':<30}{qs.stats.cagr(pnl_series_no_sl) / abs(qs.stats.max_drawdown(pnl_series_no_sl)):>20.2f}")
        print(f"{'Number of Trades':<30}{num_trades:>20}")
        print("-"*50 + "\n")
        print("="*80 + "\n")


In [ ]:

import logging
import matplotlib.font_manager
import pandas as pd
import matplotlib.pyplot as plt
import quantstats as qs

logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

filtered_seasons = best_seasons_df[
    (best_seasons_df['Period'] == 'weekday') &
    (best_seasons_df["ProfitFactor"] > 1.5) &
    (best_seasons_df["SharpeRatio"] > 1.5) &
    (best_seasons_df["NumTrades"] > 100)
].sort_values(by="ProfitFactor", ascending=False)  # Ordenado por Period y luego por Value

long = True
stop_loss_pct = sl_pct  # O asigna None si no deseas usar SL

# Definir porcentaje de uso del pct_range para TP y SL
tp_pct = 0.50  # 40% del tamaño de pct_range
sl_pct = 0.50  # 40% del tamaño de pct_range

# Iterar sobre cada combinación filtrada en orden cronológico
for index, row in filtered_seasons.iterrows():
    symbol = row["Symbol"]
    period = row["Period"]
    value = row["Value"]

    asset = qb.AddEquity(symbol, Resolution.DAILY, dataNormalizationMode=DataNormalizationMode.RAW).Symbol
    history = qb.History(asset, start=datetime(1997, 1, 1), end=qb.time, resolution=Resolution.DAILY)
    if isinstance(history.index, pd.MultiIndex):
        history = history.droplevel([0])

    if not history.empty:
        history = history[['open', 'high', 'low', 'close', 'volume']].copy()

        history['returns_pct'] = ((history['close'] - history['open']) / history['open']) * 100
        history['pct_range'] = abs((history['high'] - history['low']) / history['low']) * 100
        history['pct_range'] = history['pct_range'].shift(1)  # Usar el pct_range de la vela anterior

        if not long:
            history['returns_pct'] *= -1  # Invertir retornos para shorts

        history['month'] = history.index.get_level_values('time').month
        history['day'] = history.index.get_level_values('time').day
        history['weekday'] = history.index.get_level_values('time').weekday
       # history['hour'] = history.index.get_level_values('time').hour
        up_trend = (history['close'].shift(1) > history['close'].shift(2))
        up_breakout = (history['close'].shift(1) > history['high'].shift(2))
        down_trend = (history['close'].shift(1) < history['close'].shift(2))
        down_breakout = (history['close'].shift(1) < history['low'].shift(2))

        # Filtrar datos según los valores de period y value de la combinación actual
        filtro = history[history[period] == value].copy()

        # TAKE PROFIT (TP) - Basado en un porcentaje de pct_range
        filtro['take_profit_price'] = filtro['open'] * (1 + (tp_pct * filtro['pct_range'] / 100))
        filtro['hit_tp'] = filtro['high'] >= filtro['take_profit_price']
        filtro['returns_pct'] = np.where(filtro['hit_tp'], tp_pct * filtro['pct_range'], filtro['returns_pct'])

        # STOP LOSS (SL) - Basado en un porcentaje de pct_range (opcional)
        if stop_loss_pct is not None:
            filtro['stop_loss_price'] = filtro['open'] * (1 - (sl_pct * filtro['pct_range'] / 100))
            filtro['hit_sl'] = filtro['low'] <= filtro['stop_loss_price']
            filtro['returns_pct'] = np.where(filtro['hit_sl'], -sl_pct * filtro['pct_range'], filtro['returns_pct'])

        # Calcular equity curve con los retornos ajustados
        filtro['equity'] = (1 + filtro['returns_pct'] / 100).cumprod() * 100

        # Generar informe de estrategia
        qs.extend_pandas()
        pnls = pd.Series(filtro['returns_pct'].values, index=filtro.index) / 100
        pnls.plot_snapshot(title=f'Rendimiento de la Estrategia ({symbol}, {period} {value})', show=True)
